In [1]:
import numpy as np
import pandas as pd

import lightgbm as lgb
import pickle

from collections import defaultdict

In [5]:
# Import my modules.
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append(str(current_dir) + '/../')

from modules import utils
from modules import models
from modules import preprocess

import importlib
for m in [utils, models, preprocess]:
    importlib.reload(m)

In [6]:
df2020 = utils.load_census()
df2020

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (102) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


,adult_2017,cfips,adult_2018,adult_2019,adult_2020,adult_2021
0,41527,1001,41831,42175,42496.0,44438.0
1,158361,1003,162430,166595,171296.0,178105.0
2,20687,1005,20346,20054,19804.0,19995.0
3,17852,1007,17868,17862,17790.0,17800.0
4,44102,1009,44177,44292,44383.0,45201.0
...,...,...,...,...,...,...
3215,42989,72145,42353,41843,41383.0,44604.0
3216,7051,72147,6985,6946,6884.0,6760.0
3217,18113,72149,17809,17547,17344.0,17811.0
3218,27598,72151,27227,27029,26803.0,25533.0


In [172]:
df_train, df_test, df_subm = utils.load_dataset()
df_all = utils.merge_dataset(df_train, df_test)

In [207]:
df_train

,cfips,county,state,first_day_of_month,microbusiness_density,active,Unemployment_rate_2019,Unemployment_rate_2020,Unemployment_rate_2021,uprate1,uprate2,uprate3
row_id,,,,,,,,,,,,
1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2.9,5.4,2.8,1.862069,0.518519,0.965517
1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2.9,5.4,2.8,1.862069,0.518519,0.965517
1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2.9,5.4,2.8,1.862069,0.518519,0.965517
1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2.9,5.4,2.8,1.862069,0.518519,0.965517
1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2.9,5.4,2.8,1.862069,0.518519,0.965517
...,...,...,...,...,...,...,...,...,...,...,...,...
56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,3.0,3.8,3.2,1.266667,0.842105,1.066667
56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,3.0,3.8,3.2,1.266667,0.842105,1.066667
56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,3.0,3.8,3.2,1.266667,0.842105,1.066667


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (102) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


,GEO_ID,NAME,S0101_C01_026E
0,Geography,Geographic Area Name,Estimate!!Total!!Total population!!SELECTED AG...
1,0500000US01001,"Autauga County, Alabama",42496
2,0500000US01003,"Baldwin County, Alabama",171296
3,0500000US01005,"Barbour County, Alabama",19804
4,0500000US01007,"Bibb County, Alabama",17790
...,...,...,...
3217,0500000US72145,"Vega Baja Municipio, Puerto Rico",41383
3218,0500000US72147,"Vieques Municipio, Puerto Rico",6884
3219,0500000US72149,"Villalba Municipio, Puerto Rico",17344
3220,0500000US72151,"Yabucoa Municipio, Puerto Rico",26803


In [ ]:
df2020 = df2020.iloc[1:]
df2020['S0101_C01_026E'] = df2020['S0101_C01_026E'].astype('int')